# Circuit-centric quantum classifier
Sebastian Molina [smolinad@unal.edu.co](mailto:smolinad@unal.edu.co)

The following notebook shows the results of the variational classifier proposed in 'Circuit-centric quantum classifiers' by Schuld, et al (2018).

In [3]:
"""
Code generated with chatgpt
"""

# Check the current TensorFlow version
import tensorflow as tf
print("Current TensorFlow version:", tf.__version__)

# Uninstall the current version (optional)
!pip uninstall -y tensorflow

# Install a specific version of TensorFlow
!pip install tensorflow==2.8.0

# Verify the installation
import tensorflow as tf
print("New TensorFlow version:", tf.__version__)

Current TensorFlow version: 2.17.0
Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-da

New TensorFlow version: 2.17.0


In [1]:
!pip install tensorcircuit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 14.6 MB/s eta 0:00:00


In [2]:
import tensorcircuit as tc
import numpy as np
import tensorflow as tf

K = tc.set_backend("tensorflow")
tc.set_dtype("complex128")

class VariationalClassifier():
    """
    Defines a variational circuit in the way of 'Circuit-centric quantum classifiers' by Schuld, et al (2018).
    """
    def __init__(self, nqubits:int=10, rangeList:list=[1,2,3]):
        self.circuit = None


        self.nqubits = nqubits
        print(self.nqubits)
        self.rangeList = rangeList
        self.stepList = [self.nqubits//np.gcd(self.nqubits, c) for c in self.rangeList]+ [1]

        self.model = tf.keras.Sequential([
            tc.keras.QuantumLayer(
                self.layer,
                weights_shape=[len(self.rangeList)+1, self.nqubits + max(self.stepList) +1, 3]
            )
        ])

    """
    Given a list of ranges —see Schuld, et al. (2018)—, creates the variational circuits and its corresponding layers.
    """
    def layer(self, x, weights):

        self.circuit = tc.Circuit(self.nqubits, inputs=x)

        # Iterates over the list of "jump" ranges
        for k in range(len(self.rangeList)):
            # Applies the first iteration of U-gates.
            for i in range(self.nqubits):
                self.circuit.u(
                    i,
                    theta=weights[k,i,0],
                    phi=weights[k,i,1],
                    lbd=weights[k,i,2]
                )

            steps = self.nqubits//np.gcd(self.nqubits, self.rangeList[k])
            q_i = 0
            # Applies the iteration of controlled U-gates with the corresponding range, i.e, the index of the qubit to be controlled. For instance, with range 3, starting by q_0, the controlled qubit is q_5. After that, q_5 controls q_2, and so on.
            for j in range(1, steps+1):
                next_q_i = (q_i - self.rangeList[k]) % self.nqubits
                self.circuit.cu(
                    q_i,
                    next_q_i,
                    theta=weights[k,self.nqubits+j,0],
                    phi=weights[k,self.nqubits+j,1],
                    lbd=weights[k,self.nqubits+j,2]
                )
                q_i = next_q_i
        # Applies the 'bias' gate, previous to measuring the circuit.
        self.circuit.u(
            0,
            theta=weights[len(self.rangeList),0,0],
            phi=weights[len(self.rangeList),0,1],
            lbd=weights[len(self.rangeList),0,2]
        )

        # Measurement
        outputs = K.stack(
            [K.real(self.circuit.expectation([tc.gates.z(), [i]])) for i in range(self.nqubits)] +
            [K.real(self.circuit.expectation([tc.gates.x(), [i]])) for i in range(self.nqubits)])
        outputs = K.reshape(outputs, [-1])
        return K.sigmoid(K.sum(outputs))

    def fit(self, x_train, y_train, **kwargs):
        self.model.compile(
            loss=tf.keras.losses.Hinge(),
            optimizer=tf.keras.optimizers.Adam(0.001), # original 0.01
            metrics=["binary_accuracy"]
        )

        self.model.fit(x_train, y_train, **kwargs)


In [3]:
K = tc.set_backend("tensorflow")
tc.set_dtype("complex128")

('complex128', 'float64')

## Amplitude encoding
In order to use this circuit efficiently, as proposed in the paper, one should encode data with amplitude encoding. The implementation of amplitude encoding was taken from [here](https://tensorcircuit.readthedocs.io/en/latest/tutorials/qml_scenarios.html?highlight=mnist).

In [4]:
# NOTE: Change to the dataset used in Farhi implementation. This was done by the means of testing.
(x_train, y_train), (x_test, y_test) = tc.templates.dataset.mnist_pair_data(
    3, 6, loader=None
)

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)

In [6]:
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

In [7]:
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)

## Variational classifier

The variational classifier receives a `rangeList`, which corresponds of the range of controlled gates.

![Variational Classifier](vc.png)

In [8]:
vc = VariationalClassifier(rangeList=[1])

10


In [11]:
vc.fit(x_train_q, y_train, batch_size=32, epochs=12, validation_split=0.8)

Epoch 1/5
76/76 [==============================] - 87s 1s/step - loss: 0.7506 - binary_accuracy: 0.9700 - val_loss: 0.7412 - val_binary_accuracy: 0.9729
Epoch 2/5
76/76 [==============================] - 76s 1s/step - loss: 0.7423 - binary_accuracy: 0.9778 - val_loss: 0.7345 - val_binary_accuracy: 0.9751
Epoch 3/5
76/76 [==============================] - 73s 971ms/step - loss: 0.7363 - binary_accuracy: 0.9774 - val_loss: 0.7295 - val_binary_accuracy: 0.9766
Epoch 4/5
76/76 [==============================] - 73s 970ms/step - loss: 0.7317 - binary_accuracy: 0.9807 - val_loss: 0.7254 - val_binary_accuracy: 0.9781
Epoch 5/5
76/76 [==============================] - 82s 1s/step - loss: 0.7279 - binary_accuracy: 0.9827 - val_loss: 0.7219 - val_binary_accuracy: 0.9788


In [18]:
from sklearn.metrics import accuracy_score

y_pred = vc.model.predict(x_test)
accuracy_score(y_test, y_pred > 0.5)

0.9730691056910569